In [1]:
# 데이터 로드 및 전처리
# 필요한 도구들 임포트
import numpy as np
import pandas as pd
import re
import shutil
import os
import unicodedata
import urllib3
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# 데이터 로드
http = urllib3.PoolManager()
url ='http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:       
    shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

In [3]:
num_samples = 33000 # 사용할 샘플의 개수(33000개) 변수에 저장

In [4]:
# 전처리 함수들을 구현
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')

def preprocess_sentence(sent):
    # 위에서 구현한 함수를 내부적으로 호출
    sent = unicode_to_ascii(sent.lower())

    # 단어와 구두점 사이에 공백을 만듭니다.
    # Ex) "he is a boy." => "he is a boy ."
    sent = re.sub(r"([?.!,¿])", r" \1", sent)

    # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환합니다.
    sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)

    sent = re.sub(r"\s+", " ", sent)
    return sent

In [5]:
# 전처리 테스트
en_sent = u"Have you had dinner?"
fr_sent = u"Avez-vous déjà diné?"
print(preprocess_sentence(en_sent))
print(preprocess_sentence(fr_sent).encode('utf-8'))

have you had dinner ?
b'avez vous deja dine ?'


In [8]:
'''
전체 데이터에서 33,000개의 샘플만 불러오되, 모든 전처리를 수행하는 함수를 만듭니다.
또한 훈련 과정에서 교사 강요(Teacher Forcing)을 사용할 예정이므로,
훈련 시 사용할 디코더의 입력 시퀀스와 실제값에 해당되는 출력 시퀀스를 따로 분리하여 저장합니다.
입력 시퀀스에는 시작을 의미하는 토큰인 <sos>를 추가하고, 출력 시퀀스에는 종료를 의미하는 토큰인 <eos>를 추가합니다.
'''
def load_preprocessed_data():
    encoder_input, decoder_input, decoder_target = [], [], []

    with open("fra.txt", "r", encoding='utf8') as lines:
        for i, line in enumerate(lines):

            # source 데이터와 target 데이터 분리
            src_line, tar_line, _ = line.strip().split('\t')

            # source 데이터 전처리
            src_line_input = [w for w in preprocess_sentence(src_line).split()]

            # target 데이터 전처리
            tar_line = preprocess_sentence(tar_line)
            tar_line_input = [w for w in ("<sos> " + tar_line).split()]
            tar_line_target = [w for w in (tar_line + " <eos>").split()]

            encoder_input.append(src_line_input)
            decoder_input.append(tar_line_input)
            decoder_target.append(tar_line_target)

            if i == num_samples - 1:
                break

    return encoder_input, decoder_input, decoder_target

In [9]:
# 얻은 3개의 데이터셋인  인코더의 입력, 디코더의 입력, 디코더의 실제값을 상위 5개 샘플만 출력해보기
sents_en_in, sents_fra_in, sents_fra_out = load_preprocessed_data()
print(sents_en_in[:5])
print(sents_fra_in[:5])
print(sents_fra_out[:5])

[['go', '.'], ['hi', '.'], ['hi', '.'], ['run', '!'], ['run', '!']]
[['<sos>', 'va', '!'], ['<sos>', 'salut', '!'], ['<sos>', 'salut', '.'], ['<sos>', 'cours', '!'], ['<sos>', 'courez', '!']]
[['va', '!', '<eos>'], ['salut', '!', '<eos>'], ['salut', '.', '<eos>'], ['cours', '!', '<eos>'], ['courez', '!', '<eos>']]


In [10]:
# 케라스 토크나이저를 통해 단어 집합을 생성하고, 텍스트 시퀀스를 정수 시퀀스로 변환하는 정수 인코딩 과정을 수행
tokenizer_en = Tokenizer(filters="", lower=False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)

tokenizer_fra = Tokenizer(filters="", lower=False)
tokenizer_fra.fit_on_texts(sents_fra_in)
tokenizer_fra.fit_on_texts(sents_fra_out)
decoder_input = tokenizer_fra.texts_to_sequences(sents_fra_in)
decoder_target = tokenizer_fra.texts_to_sequences(sents_fra_out)

In [11]:
# 패딩 수행
encoder_input = pad_sequences(encoder_input, padding="post")
decoder_input = pad_sequences(decoder_input, padding="post")
decoder_target = pad_sequences(decoder_target, padding="post")

In [14]:
# 얻은 데이터의 크기(shape)을 확인
print(encoder_input.shape)
print(decoder_input.shape)
print(decoder_target.shape)

(33000, 8)
(33000, 16)
(33000, 16)


In [15]:
# 단어 집합의 크기를 정의
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_fra.word_index) + 1
print("영어 단어 집합의 크기 : {:d}, 프랑스어 단어 집합의 크기 : {:d}".format(src_vocab_size, tar_vocab_size))

영어 단어 집합의 크기 : 4678, 프랑스어 단어 집합의 크기 : 8032


In [16]:
'''
단어로부터 정수를 얻는 딕셔너리와 정수로부터 단어를 얻는 딕셔너리를 각각 만들어줍니다.
이들은 훈련을 마치고 예측 과정과 실제값과 결과를 비교하는 경우에 사용됩니다.
'''
src_to_index = tokenizer_en.word_index
index_to_src = tokenizer_en.index_word # 훈련 후 결과 비교할 때 사용

tar_to_index = tokenizer_fra.word_index # 훈련 후 예측 과정에서 사용
index_to_tar = tokenizer_fra.index_word # 훈련 후 결과 비교할 때 사용

In [17]:
# 테스트 데이터를 분리하기 전에, 적절한 분포를 갖도록 데이터를 섞어주는 과정을 진행
# 이를 위해 우선 순서가 섞인 정수 시퀀스 리스트를 만든다.
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print(indices)

[19764 17230  6028 ... 15899  3552 28968]


In [18]:
# 이를 데이터셋의 순서로 지정
# 샘플들이 기존 순서와 다른 순서로 섞이게 됨
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [20]:
# 임의로 30,997번째 샘플을 출력
encoder_input[30997]

array([ 2, 39, 16, 46,  1,  0,  0,  0])

In [21]:
decoder_input[30997]

array([  2,  11,  50, 619,  22,   1,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0])

In [22]:
decoder_target[30997]

array([ 11,  50, 619,  22,   1,   3,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0])

In [23]:
# 훈련 데이터의 10%를 테스트 데이터로 분리
n_of_val = int(33000*0.1)
print(n_of_val)

3300


In [25]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print(encoder_input_train.shape) # 훈련 데이터의 샘플은 29,700개
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)  # 테스트 데이터의 샘플은 3,300개
print(decoder_input_test.shape)
print(decoder_target_test.shape)

(29700, 8)
(29700, 16)
(29700, 16)
(3300, 8)
(3300, 16)
(3300, 16)


In [26]:
# 기계 번역기 만들기
# 모델 설계를 위해 필요한 도구들을 임포트
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

# 임베딩 벡터와 LSTM의 은닉 상태의 크기를 특정 크기로 고정. 여기서는 50.
latent_dim = 50

In [27]:
# 인코더 설계
# Masking은 패딩 토큰인 숫자 0의 경우에는 연산을 제외하는 역할을 수행
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(src_vocab_size, latent_dim)(encoder_inputs) # 임베딩 층
enc_masking = Masking(mask_value=0.0)(enc_emb) # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(latent_dim, return_state=True) # 상태값 리턴을 위해 return_state는 True
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking) # 은닉 상태와 셀 상태를 리턴
encoder_states = [state_h, state_c] # 인코더의 은닉 상태와 셀 상태를 저장

In [28]:
# 디코더 설계
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, latent_dim) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) # 패딩 0은 연산에서 제외
dec_masking = Masking(mask_value=0.0)(dec_emb)

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True) 

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                     initial_state=encoder_states)

# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [29]:
# 모델의 입출력을 정의
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])

# 모델의 파라미터 확인
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 50)     233900      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     401600      input_2[0][0]                    
_______________________________________________________________________________________

In [30]:
# 모델 훈련
model.fit(x = [encoder_input_train, decoder_input_train], y = decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size = 128, epochs = 50)

Epoch 1/50
233/233 [==============================] - 141s 604ms/step - loss: 3.1584 - acc: 0.6174 - val_loss: 1.8745 - val_acc: 0.7142
Epoch 2/50
233/233 [==============================] - 137s 590ms/step - loss: 1.6939 - acc: 0.7328 - val_loss: 1.6115 - val_acc: 0.7396
Epoch 3/50
233/233 [==============================] - 132s 566ms/step - loss: 1.5157 - acc: 0.7519 - val_loss: 1.4910 - val_acc: 0.7572
Epoch 4/50
233/233 [==============================] - 124s 534ms/step - loss: 1.4126 - acc: 0.7719 - val_loss: 1.4230 - val_acc: 0.7755
Epoch 5/50
233/233 [==============================] - 125s 538ms/step - loss: 1.3367 - acc: 0.7844 - val_loss: 1.3620 - val_acc: 0.7847
Epoch 6/50
233/233 [==============================] - 125s 537ms/step - loss: 1.2752 - acc: 0.7938 - val_loss: 1.3104 - val_acc: 0.7902
Epoch 7/50
233/233 [==============================] - 131s 564ms/step - loss: 1.2247 - acc: 0.8011 - val_loss: 1.2709 - val_acc: 0.7958
Epoch 8/50
233/233 [============================

In [31]:
# seq2seq 기계 번역기 동작시키기
'''
seq2seq는 훈련 과정과 테스트 과정에서의 동작 방식이 다릅니다.
그래서 테스트 과정을 위해 모델을 다시 설계해주어야 합니다. 
'''

# 인코더 설계
encoder_model = Model(encoder_inputs, encoder_states)

# 디코더 설계
# 이전 시점의 상태를 보관할 텐서
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2= dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단어 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

# 디코더 정의
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [32]:
# 테스트 과정에서의 동작을 위한 함수 구현
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 정수 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = tar_to_index['<sos>']

    stop_condition = False
    decoded_sentence = ''

    # stop_condition이 True가 될 때까지 루프 반복
    # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 단어로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

         # 현재 시점의 예측 단어를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
        if (sampled_char == '<eos>' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [33]:
# 결과 확인을 위한 함수 정의
def seq2src(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + index_to_src[i]+' '
    return temp

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2tar(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=tar_to_index['<sos>']) and i!=tar_to_index['<eos>']):
            temp = temp + index_to_tar[i] + ' '
    return temp

In [35]:
# 훈련 데이터에 대해서 임의로 선택한 인덱스의 샘플의 결과를 출력
for seq_index in [176, 276, 376, 476, 576, 676, 776, 876, 976, 1076]:
  input_seq = encoder_input_train[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("원문 : ",seq2src(encoder_input_train[seq_index]))
  print("번역문 :",seq2tar(decoder_input_train[seq_index]))
  print("예측문 :",decoded_sentence[:-5])
  print("\n")

원문 :  i paid the bill . 
번역문 : j ai paye l addition . 
예측문 :  j ai pris le chien . 


원문 :  i m bilingual . 
번역문 : je suis bilingue . 
예측문 :  je suis une fois . 


원문 :  please get tom . 
번역문 : allez chercher tom s il vous plait . 
예측문 :  veuillez faire tout ou tom . 


원문 :  we re ready now . 
번역문 : nous sommes prets desormais . 
예측문 :  nous sommes desormais pretes . 


원문 :  that s a lemon tree . 
번역문 : c est un citronnier . 
예측문 :  c est un bon j . 


원문 :  are you envious ? 
번역문 : etes vous jalouse ? 
예측문 :  etes vous des enfants ? 


원문 :  are you ambitious ? 
번역문 : etes vous ambitieuse ? 
예측문 :  es tu seul ? 


원문 :  i like what you did . 
번역문 : j apprecie ce que vous avez fait . 
예측문 :  tu sais que tu as tort . 


원문 :  i have no choice . 
번역문 : je n ai pas le choix . 
예측문 :  je n ai pas le choix . 


원문 :  is tom lazy ? 
번역문 : est ce que tom est paresseux ? 
예측문 :  est ce que tom est libre ? 




In [36]:
# 테스트 데이터에 대해서 임의로 선택한 인덱스의 샘플의 결과를 출력
for seq_index in [176, 276, 376, 476, 576, 676, 776, 876, 976, 1076]:
  input_seq = encoder_input_test[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("원문 : ",seq2src(encoder_input_test[seq_index]))
  print("번역문 :",seq2tar(decoder_input_test[seq_index]))
  print("예측문 :",decoded_sentence[:-5])
  print("\n")

원문 :  it was clean . 
번역문 : c etait propre . 
예측문 :  c etait de l amour . 


원문 :  that was unexpected . 
번역문 : ce n etait pas prevu . 
예측문 :  c etait de l amour . 


원문 :  i m buying . 
번역문 : c est moi qui paie . 
예측문 :  je vais ferai . 


원문 :  just wait a second . 
번역문 : attends juste une seconde . 
예측문 :  attendez une seconde ! 


원문 :  can i eat this cake ? 
번역문 : puis je manger ce gateau ? 
예측문 :  puis je manger ce livre ? 


원문 :  does he have money ? 
번역문 : a t il de l argent ? 
예측문 :  est ce que elle a l qu il sont ? 


원문 :  stop harassing me . 
번역문 : arretez de me harceler . 
예측문 :  arrete de me fut yeux ! 


원문 :  he ll get over it . 
번역문 : il s en remettra . 
예측문 :  il va vous vie . 


원문 :  fill in the blanks . 
번역문 : remplissez les blancs . 
예측문 :  les amour . 


원문 :  it s booby trapped . 
번역문 : c est piege . 
예측문 :  c est une chanson . 


